# Lab 10: Incident Response Copilot

Build an AI copilot that assists with incident response investigations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab10_ir_copilot.ipynb)

## Learning Objectives
- Conversational IR workflow
- State management for investigations
- Tool-assisted data gathering
- Playbook execution guidance

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass, field
from datetime import datetime
import json

## 1. Investigation State

In [ ]:
@dataclass
class IRState:
    """Incident response investigation state."""
    incident_id: Optional[str] = None
    incident_type: Optional[str] = None
    affected_hosts: List[str] = field(default_factory=list)
    investigated_iocs: List[Dict] = field(default_factory=list)
    actions_taken: List[Dict] = field(default_factory=list)
    timeline: List[Dict] = field(default_factory=list)
    containment_status: str = "none"
    messages: List[Dict] = field(default_factory=list)

class StateManager:
    """Manage investigation state."""
    
    def __init__(self):
        self.state = IRState()
    
    def set_incident(self, incident_id: str, incident_type: str):
        self.state.incident_id = incident_id
        self.state.incident_type = incident_type
        self.add_timeline("Investigation started")
    
    def add_host(self, hostname: str):
        if hostname not in self.state.affected_hosts:
            self.state.affected_hosts.append(hostname)
    
    def add_ioc(self, ioc: str, result: Dict):
        self.state.investigated_iocs.append({
            "ioc": ioc,
            "result": result,
            "timestamp": datetime.now().isoformat()
        })
    
    def add_action(self, action: str, result: str):
        self.state.actions_taken.append({
            "action": action,
            "result": result,
            "timestamp": datetime.now().isoformat()
        })
        self.add_timeline(action)
    
    def add_timeline(self, event: str):
        self.state.timeline.append({
            "event": event,
            "timestamp": datetime.now().isoformat()
        })
    
    def add_message(self, role: str, content: str):
        self.state.messages.append({"role": role, "content": content})

## 2. Copilot Tools

In [ ]:
# Mock security data
MOCK_DATA = {
    "hosts": {
        "WORKSTATION-42": {
            "hostname": "WORKSTATION-42",
            "ip": "192.168.1.42",
            "os": "Windows 10",
            "user": "jsmith",
            "last_seen": "2024-01-15T09:30:00Z"
        }
    },
    "iocs": {
        "185.143.223.47": {
            "classification": "Malicious",
            "threat_type": "C2 Server",
            "malware": "Cobalt Strike"
        }
    }
}

class CopilotTools:
    """Tools available to the IR copilot."""
    
    def __init__(self, state_manager: StateManager):
        self.state = state_manager
        self.isolated_hosts = []
    
    def get_host_info(self, hostname: str) -> Dict:
        """Get information about a host."""
        if hostname in MOCK_DATA["hosts"]:
            self.state.add_host(hostname)
            return MOCK_DATA["hosts"][hostname]
        return {"error": f"Host {hostname} not found"}
    
    def lookup_ioc(self, ioc: str) -> Dict:
        """Look up IOC in threat intelligence."""
        result = MOCK_DATA["iocs"].get(ioc, {"classification": "Unknown"})
        self.state.add_ioc(ioc, result)
        return result
    
    def isolate_host(self, hostname: str, confirm: bool = False) -> Dict:
        """Isolate a host from network."""
        if not confirm:
            return {
                "requires_confirmation": True,
                "message": f"Confirm isolation of {hostname}?"
            }
        
        self.isolated_hosts.append(hostname)
        self.state.add_action(f"Isolated host {hostname}", "Success")
        self.state.state.containment_status = "partial"
        return {"success": True, "message": f"{hostname} isolated"}

# Test tools
manager = StateManager()
tools = CopilotTools(manager)

print("Host Info:")
print(json.dumps(tools.get_host_info("WORKSTATION-42"), indent=2))

print("\nIOC Lookup:")
print(json.dumps(tools.lookup_ioc("185.143.223.47"), indent=2))

## 3. IR Copilot

In [ ]:
class IRCopilot:
    """AI-powered incident response copilot."""
    
    def __init__(self):
        self.state_manager = StateManager()
        self.tools = CopilotTools(self.state_manager)
        try:
            from anthropic import Anthropic
            self.client = Anthropic()
            self.available = True
        except:
            self.available = False
    
    def chat(self, message: str) -> str:
        """Process user message and respond."""
        self.state_manager.add_message("user", message)
        
        if not self.available:
            response = self._process_local(message)
        else:
            response = self._process_with_llm(message)
        
        self.state_manager.add_message("assistant", response)
        return response
    
    def _process_local(self, message: str) -> str:
        """Process message without LLM."""
        msg_lower = message.lower()
        
        if "workstation" in msg_lower:
            info = self.tools.get_host_info("WORKSTATION-42")
            return f"Host info retrieved:\n{json.dumps(info, indent=2)}"
        
        if "185.143" in message or "ioc" in msg_lower:
            result = self.tools.lookup_ioc("185.143.223.47")
            return f"IOC lookup result:\n{json.dumps(result, indent=2)}"
        
        if "isolate" in msg_lower:
            return "To isolate a host, please confirm the action. Type 'confirm isolate HOSTNAME'."
        
        if "status" in msg_lower:
            return self.get_status()
        
        return "I can help with: host lookups, IOC analysis, containment actions, and investigation status."
    
    def _process_with_llm(self, message: str) -> str:
        """Process with LLM and tools."""
        # Simplified - in production, use tool_use
        return self._process_local(message)
    
    def get_status(self) -> str:
        """Get investigation status."""
        s = self.state_manager.state
        return f"""
## Investigation Status

- **Incident**: {s.incident_id or 'Not set'}
- **Type**: {s.incident_type or 'Unknown'}
- **Affected Hosts**: {len(s.affected_hosts)}
- **IOCs Investigated**: {len(s.investigated_iocs)}
- **Containment**: {s.containment_status}

### Timeline
""" + "\n".join([f"- {e['event']}" for e in s.timeline[-5:]])

In [ ]:
# Interactive session
copilot = IRCopilot()
copilot.state_manager.set_incident("INC-2024-001", "Malware")

# Simulate conversation
queries = [
    "What happened on WORKSTATION-42?",
    "Look up IOC 185.143.223.47",
    "What's the investigation status?"
]

for q in queries:
    print(f"\nUser: {q}")
    print("-" * 40)
    response = copilot.chat(q)
    print(response)

## 4. Playbook Executor

In [ ]:
PLAYBOOKS = {
    "malware": [
        {"step": 1, "action": "Identify affected hosts", "auto": True},
        {"step": 2, "action": "Isolate from network", "auto": False},
        {"step": 3, "action": "Collect forensic evidence", "auto": False},
        {"step": 4, "action": "Analyze malware sample", "auto": True},
        {"step": 5, "action": "Block IOCs", "auto": True},
        {"step": 6, "action": "Remediate affected systems", "auto": False}
    ]
}

class PlaybookExecutor:
    """Execute IR playbooks."""
    
    def __init__(self, copilot: IRCopilot):
        self.copilot = copilot
        self.current_playbook = None
        self.current_step = 0
    
    def start_playbook(self, playbook_name: str) -> str:
        if playbook_name not in PLAYBOOKS:
            return f"Unknown playbook: {playbook_name}"
        
        self.current_playbook = playbook_name
        self.current_step = 0
        return self.get_next_step()
    
    def get_next_step(self) -> str:
        if not self.current_playbook:
            return "No playbook active"
        
        steps = PLAYBOOKS[self.current_playbook]
        if self.current_step >= len(steps):
            return "Playbook complete!"
        
        step = steps[self.current_step]
        auto = "[AUTO]" if step["auto"] else "[MANUAL]"
        return f"Step {step['step']}: {auto} {step['action']}"
    
    def complete_step(self) -> str:
        self.current_step += 1
        self.copilot.state_manager.add_timeline(
            f"Completed playbook step {self.current_step}"
        )
        return self.get_next_step()

# Test playbook
executor = PlaybookExecutor(copilot)
print(executor.start_playbook("malware"))
print(executor.complete_step())
print(executor.complete_step())

## Summary

We built an IR copilot with:

1. **State Management** - Track investigation progress
2. **Tools** - Host info, IOC lookup, containment
3. **Conversational Interface** - Natural language interaction
4. **Playbook Execution** - Guided response workflows

### Next Steps:
1. Add more data source integrations
2. Implement automated evidence collection
3. Create incident documentation generator